# Clause Level Marijuana Legalization Annotation 

[Google Doc](https://docs.google.com/document/d/1kuDieHGpZoVCDrCeGPiCQhd6BoGQM9hjexFGF_lFOXU/edit?ts=5fecb209)

#TODO

0. Train aspect on both sitent and reddit

1. need to parse documents into clauses

# Models 
1. 2 models for genericity - one on sitent, one on reddit. with as good transfer as we can. 64 bs, 1 epoch, class weights
2. doesn't matter a lot for aspect since transfer is good. 
3. Boundedness - reddit train, 32 batch size, 2 epochs, temporal features

# Importing and setting up stuff

In [ ]:
from google.colab import drive
import os 

import tensorflow as tf
print(tf.__version__)

!pip install transformers
from transformers import RobertaTokenizer, TFRobertaModel

import spacy

!pip install pytokenizations==0.7.2
import tokenizations

!pip install numpy 
from numpy import asarray
from numpy import savetxt, loadtxt
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers

import xml.etree.ElementTree as ET 

from keras import backend as K

from sklearn.model_selection import train_test_split

import h5py

from tensorflow.keras.models import load_model
from sklearn.metrics import precision_recall_fscore_support, classification_report

# sutime stuff
!pip install sutime
from sutime import SUTime
import json
!apt-get install maven
!mvn dependency:copy-dependencies -DoutputDirectory=./jars -f $(python -c 'import importlib; import pathlib; print(pathlib.Path(importlib.util.find_spec("sutime").origin).parent / "pom.xml")')

# oversampling and classweights and SMOTE and stuff 
from sklearn.utils import class_weight
from imblearn.over_sampling import RandomOverSampler

2.5.0
     |████████████████████████████████| 2.6 MB 31.3 MB/s 
     |████████████████████████████████| 636 kB 50.6 MB/s 
     |████████████████████████████████| 895 kB 43.7 MB/s 
     |████████████████████████████████| 3.3 MB 44.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


     |████████████████████████████████| 240 kB 36.2 MB/s 
     |████████████████████████████████| 448 kB 47.7 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libaopalliance-java libapache-pom-java libatinject-jsr330-api-java
  libcdi-api-java libcommons-cli-java libcommons-io-java libcommons-lang3-java
  libcommons-parent-java libgeronimo-annotation-1.3-spec-java
  libgeronimo-interceptor-3.0-spec-java libguava-java libguice-java
  libhawtjni-runtime-java libjansi-java libjansi-native-java libjsr305-java
  libmaven-parent-java libmaven-resolver-java libmaven-shared-utils-java
  libmaven3-core-java libplexus-cipher-java libplexus-classworlds-java
  libplexus-component-annotations-java libplexus-interpolation-java
  libplexus-sec-dispatcher-java libplexus-utils2-java libsisu-inject-java
  libsisu-plexus-java libslf4j-java libwagon-file-java
  libwagon-http-shaded-java libwagon-pro

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


#Global variables

In [ ]:
drive.mount('/content/drive',force_remount=True)

# annotator names should match the folder names in the directory
annotators = ['Kate', 'Sheridan', 'Muskaan']

POS = True 
NE = True

MAX_CLAUSE_LENGTH = 70

TOKENIZER = RobertaTokenizer.from_pretrained("roberta-base")  
MODEL = TFRobertaModel.from_pretrained('roberta-base')

SITENT_DIRECTORY = '/content/drive/MyDrive/sitent_xml/'
REDDIT_GLOVE_DIRECTORY = '/content/drive/My Drive/Annotation_Evaluation_12.29.20/'

MODEL_NAME = 'final_reddit_sitent_genericity_64_3_cw.ckpt'
CHECKPOINT_PATH = "/content/drive/MyDrive/Clause Classification Models/saved/" + MODEL_NAME
MODEL_DIRECTORY = '/content/drive/MyDrive/Clause Classification Models'

BATCH_SIZE = 64
EPOCHS = 3

DATASET = 'sitent' # sitent or reddit
CLASSIFICATION_TYPE = 'genericity' # genericity, aspect, or boundedness

TEMPORAL = False

if CLASSIFICATION_TYPE in ['aspect', 'genericity'] and DATASET == 'sitent':
  NUM_LABELS = 3
else:
  NUM_LABELS = 2

# Spacy and corenlp stuff 
nlp = spacy.load("en_core_web_sm")
sutime = SUTime(mark_time_ranges=True)

Mounted at /content/drive


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
def get_reddit_breakdown():
    curr_labels = tf.zeros([1])
    for filename in os.listdir('/content/drive/MyDrive/Clause Classification Models'+'/inputs/reddit_glove/'):

      file_id = filename.split('.')[0].split('_')[3]
      print('reading in file ' + file_id + '...')

      if len(filename.split('.')[0].split('_')) > 4:
        file_id += '_' + filename.split('.')[0].split('_')[4]

    # read labels based on the classification task
      if CLASSIFICATION_TYPE=='genericity':
        h5f = h5py.File(MODEL_DIRECTORY+'/labels/reddit_glove_genericity/reddit_glove_genericity_labels_' + file_id + '.h5','r')
        clause_labels = h5f['genericity_labels'][:]
        h5f.close()
      elif CLASSIFICATION_TYPE=='boundedness':
        h5f = h5py.File(MODEL_DIRECTORY+'/labels/reddit_glove_boundedness/reddit_glove_boundedness_labels_' + file_id + '.h5','r')
        clause_labels = h5f['boundedness_labels'][:]
        h5f.close()     
      elif CLASSIFICATION_TYPE == 'aspect':
        h5f = h5py.File(MODEL_DIRECTORY+'/labels/reddit_glove_aspect/reddit_glove_aspect_labels_' + file_id + '.h5','r')
        clause_labels = h5f['aspect_labels'][:]
        h5f.close() 

      new_labels = tf.concat([clause_labels, curr_labels], axis=0)
      curr_labels = new_labels

      print(curr_labels.shape)
    
    zero = 0
    one = 0
    for label in curr_labels:
      if label.numpy() == 0:
        zero+=1
      elif label.numpy() == 1:
        one+=1
      else:
        'the fuck'

    print(zero)
    print(one)

# GENERIC = 9347
# SPECIFIC = 9404
# BOUNDED = 5436
# UNBOUNDED = 13315
# DYNAMIC = 10614
# STATIVE = 8137
CLASSIFICATION_TYPE = 'aspect'
get_reddit_breakdown()

reading in file 091120113028...
(225,)
reading in file 291020230505...
(294,)
reading in file 30920174438...
(345,)
reading in file 091120113041...
(407,)
reading in file 091120113025...
(458,)
reading in file 291020230442...
(500,)
reading in file 51020131148...
(522,)
reading in file 30920174446...
(579,)
reading in file 51020131203...
(606,)
reading in file 51020131202...
(682,)
reading in file 51020131206...
(749,)
reading in file 51020131151...
(819,)
reading in file 51020131153...
(877,)
reading in file 51020131143...
(958,)
reading in file 51020131156...
(1013,)
reading in file 51020131142...
(1044,)
reading in file 291020230455...
(1089,)
reading in file 291020230445...
(1153,)
reading in file 291020230450...
(1219,)
reading in file 291020230452...
(1265,)
reading in file 30920174440...
(1332,)
reading in file 091120113037...
(1358,)
reading in file 22886...
(1399,)
reading in file 41729...
(1435,)
reading in file 291020230447...
(1483,)
reading in file 291020230503...
(1511,)


#Preprocessing - 



### Getting temporal type from sutime

In [ ]:
def get_temporal_info(clause):
  ''' 
  Given a clause, this function gets a one hot representation of the temporal
  annotations in the clause. There are 4 types. DATE, TIME, DURATION, SET
  '''

  annotations = sutime.parse(clause)
  types = {'DATE':0, 'TIME':0, 'DURATION':0, 'SET':0}
  for annotation in annotations:
    if annotation['type'] == 'DATE':
      types['DATE'] = 1
    elif annotation['type'] == 'TIME':
      types['TIME'] = 1
    elif annotation['type'] == 'DURATION':
      types['DURATION'] = 1
    elif annotation['type'] == 'SET':
      types['SET'] = 1
    
  return tf.cast(tf.stack(list(types.values())), dtype='float32')

## roBERTa Features

get_one_hot_pos(pos): returns a one hot encoding for the POS tag 

get_one_hot_ne(ne): returns a one hot encoding for the NE tag 

In [ ]:
def get_one_hot_pos(pos):
  '''
  Given a POS tag, return its one hot encoding that will be concatenated to the
  word embedding
  @param pos: the POS tag
  @return: one one hot encoding of the POS tag
  '''
  POS_TAGS = {
      'ADJ': 0, 'ADP': 1, 'ADV': 2, 'AUX': 3, 'CONJ': 4, 'CCONJ': 5, 'DET': 6,
        'INTJ': 7, 'NOUN': 8, 'NUM': 9, 'PART': 10, 'PRON': 11, 'PROPN': 12,
        'PUNCT': 13, 'SCONJ': 14, 'SYM': 15, 'VERB': 16, 'X': 17, 'SPACE': 18
  }
  one_hot_matrix = tf.eye(len(POS_TAGS))
  if pos == '':
    return tf.zeros(len(POS_TAGS))
  return one_hot_matrix[POS_TAGS[pos]]

def get_one_hot_ne(ne):
  '''
  Given a NE tag, return its one hot encoding that will be concatenated to the
  word embedding
  @param NE: the NE tag
  @return: one one hot encoding of the NE tag
  '''
  NE_TAGS = { 'PERSON': 0, 'NORP': 1, 'FAC': 2, 'ORG': 3, 'GPE': 4,'LOC': 5,
                  'PRODUCT': 6, 'EVENT': 7, 'WORK_OF_ART': 8, 'LAW': 9, 
                  'LANGUAGE': 10, 'DATE': 11, 'TIME': 12, 'PERCENT': 13, 
                  'MONEY': 14, 'QUANTITY': 15, 'ORDINAL': 16, 'CARDINAL': 17
  }
  one_hot_matrix = tf.eye(len(NE_TAGS))
  if ne == '':
    return tf.zeros(len(NE_TAGS))
  return one_hot_matrix[NE_TAGS[ne]]

3. add_pos_ne_encoding(tokens, doc, vectors, pos=True, ne=True): takes in the tokens for a clause, finds out the POS and NE tags (if set to True) using the spaCy doc, and concatenated the one hot encoding to the respective word vectors. 

In [ ]:
def add_pos_ne_encoding(tokens, doc, vectors, pos=True, ne=True):
  '''
  This function takes in the tokens for a clause, the nlp doc with the tags, and the vectors.
  It concatenates the POS tag encoding to their respective word vectors.
  @param tokens:
  @param doc:
  @param vectors:
  @return: 
  '''
  # for each token, find its POS tag, get encoding, concatenate to its vector.
  # if the token doesn't have a tag, pad with 0s 

  if (not pos) and (not ne):
    return vectors

  spacy_tokens = [token.text for token in doc]
  roberta_tokens = tokens 
  a2b, b2a = tokenizations.get_alignments(spacy_tokens, roberta_tokens)

  new_vectors = []

  for index, alignment in enumerate(b2a):
    if alignment:
      # get the tags from that spacy token and concat
      if pos and ne: #both pos and ne
        new_vectors.append(tf.concat([vectors[0][index], get_one_hot_pos(doc[alignment[0]].pos_), get_one_hot_ne(doc[alignment[0]].ent_type_)], axis=0)) 
      elif ne: #ne only
        new_vectors.append(tf.concat([vectors[0][index], get_one_hot_ne(doc[alignment[0]].ent_type_)], axis=0))
      else: #pos only
        new_vectors.append(tf.concat([vectors[0][index], get_one_hot_pos(doc[alignment[0]].pos_)], axis=0))
    else:
      # concat zeros 
      if pos and ne: #both pos and ne
        new_vectors.append(tf.concat([vectors[0][index], tf.zeros([37])], 0))  
      elif ne: #ne only
        new_vectors.append(tf.concat([vectors[0][index], tf.zeros([18])], 0)) 
      else: #pos only
        new_vectors.append(tf.concat([vectors[0][index], tf.zeros([19])], 0)) 

  new_vectors = tf.stack(new_vectors)
  return new_vectors

4. make_embeddings(): makes the embeddings for each word in a clause using the tokenizer to tokenize, the model to get the hidden state, adding the POS and NE tag if set to true 



In [ ]:
def make_embeddings(clause, tokenizer, model, pos=True, ne=True):
  doc = nlp(clause)

  # tokenize and get the output from the model
  tokenized_clause = tokenizer(clause, padding='max_length', max_length=MAX_CLAUSE_LENGTH, return_tensors='tf')
  outputs = model(**tokenized_clause)

  # get the tokens from the tokenizer
  tokens = (tokenizer.convert_ids_to_tokens(tokenized_clause['input_ids'][0]))

  # add POS and NE encodings 
  vectors_with_pos_ne = add_pos_ne_encoding(tokens, doc, outputs.last_hidden_state, pos, ne)

  temporal_info_encoding = get_temporal_info(clause)
  temporal_info_encoding = (tf.repeat(tf.expand_dims(temporal_info_encoding, axis=0),[70], axis=0))
  
  return tf.concat([vectors_with_pos_ne, temporal_info_encoding], axis=1)

## Reddit Glove

5. get_labels_and_confidence(annotation)

In [ ]:
def get_genericity_aspect_boundedness_labels(annotation):
  '''
  Given an annotation string, returns the label (generic or specific) (0 or 1)
  and confidence rating (from 1 to 5) (from 0.2 to 1). Ignore confidence of 0.
  @param annotation: the annotion string from the annotated documents 
  @return genericity_label: label 0 (GENERIC) or 1 (SPECIFIC)
  @return boundedness_label: confidence rating (2, 3, 4, 5)
  '''

  genericity_label = -1
  if 'BASIC STATE' in annotation:
    genericity_label = 1
  elif 'GENERIC' in annotation or 'generic' in annotation:
    genericity_label = 0
  elif 'SPECIFIC' in annotation or 'specific' in annotation:
    genericity_label = 1
  elif 'GENERALIZING' in annotation:
    genericity_label = 0
  elif 'COERCED' in annotation and 'SPECIFIC' in annotation:
    genericity_label = 1
  elif 'COERCED' in annotation and 'GENERIC' in annotation:
    genericity_label = 0

  boundedness_label = -1
  if 'BOUNDED EVENT' in annotation:
    boundedness_label = 0
  else:
    boundedness_label = 1
  
  aspect_label = -1
  if 'DYNAMIC' in annotation:
    aspect_label = 0
  elif 'STATIVE' in annotation:
    aspect_label = 1
  elif 'COERCED' in annotation:
    aspect_label = 0
  elif 'EVENT' in annotation:
    aspect_label = 0 
  elif 'STATE' in annotation:
    aspect_label = 1 
  elif 'GENERIC' in annotation:
    aspect_label = 1 
  
  
  return genericity_label, aspect_label, boundedness_label


In [ ]:
def parse_file(file, tokenizer, model, pos=True, ne=True):
  '''
  @param file: the file that we want to parse (already opened)
  @param tokenizer: the tokenizer to use 
  @param model:
  @param pos:
  @param ne:
  '''
  lines = file.readlines()
    
  clauses = []
  genericity_labels = []
  aspect_labels = []
  boundedness_labels = []

  # for each line in the document
  for line in lines:

    # split at the annotation mark
    splited = line.split(' ##')

    # skip this line if it didnt have the ##
    if len(splited) < 2:
      continue

    clause = splited[0]
    annotation = splited[1]

    # get label and confidence
    # label, confidence = 
    genericity_label, aspect_label, boundedness_label = get_genericity_aspect_boundedness_labels(annotation)

    if genericity_label == -1 or boundedness_label == -1 or aspect_label == -1:
      # skip this line 
      continue 

    # get embeddings for the clause
    clause_embeddings = make_embeddings(clause, tokenizer, model)

    # append the clause, label, and confidence rating
    clauses.append(clause_embeddings)
    genericity_labels.append(genericity_label)
    aspect_labels.append(aspect_label)
    boundedness_labels.append(boundedness_label)

  file_embeddings = tf.stack(clauses)
  file_genericity_labels = tf.stack(genericity_labels)
  file_aspect_labels = tf.stack(aspect_labels)
  file_boundedness_labels = tf.stack(boundedness_labels)
  return file_embeddings, file_genericity_labels, file_aspect_labels, file_boundedness_labels

In [ ]:
def parse_and_save_reddit(directory):
  '''
  @param directory: string denoting the location of the directory with the annotator folders 
  @param pos: True if we want to use POS tags (true by default)
  @param ne: True if we want to use NE tags (true by default)
  '''

  # for each annotator in the list of annotators, go through each file, check 
  # if it ends with .txt, then parse it
  for annotator in annotators:
    for filename in os.listdir(directory+annotator):
      if filename.endswith('.txt'):
        file_id = filename.split('.')[0]
        curr_file = open(directory+annotator+'/'+filename)
        print('parsing ' + filename + '...')

        reddit_embeddings_directory = MODEL_DIRECTORY+'/inputs/reddit_glove/'

        if 'reddit_glove_embeddings_' + file_id + '.h5' not in os.listdir(reddit_embeddings_directory):
          # get embeddings, labels for all clauses in the file
          file_embeddings, file_genericity_labels, file_aspect_labels, file_boundedness_labels = parse_file(curr_file, TOKENIZER, MODEL, POS, NE)

          # save the embeddings and labels 
          save_data(file_embeddings, file_genericity_labels, file_aspect_labels, file_boundedness_labels, file_id)

          print(file_id + ' saved! - ', file_embeddings.shape, file_genericity_labels.shape, file_boundedness_labels.shape)
        else:
          print('already saved!')
  print('done')

parse_and_save_reddit(REDDIT_GLOVE_DIRECTORY)

parsing 091120113028.txt...
already saved...
parsing 291020230505.txt...
already saved...
parsing 291020230442.txt...
already saved...
parsing 11776.txt...
already saved...
parsing 30920174441.txt...
already saved...
parsing 291020230448.txt...
already saved...
parsing 51020131206.txt...
already saved...
parsing 291020230455.txt...
already saved...
parsing 291020230450.txt...
already saved...
parsing 291020230452.txt...
already saved...
parsing 091120113037.txt...
already saved...
parsing 291020230447.txt...
already saved...
parsing 050121173108.txt...
already saved...
parsing 050121173101.txt...
already saved...
parsing 050121172355.txt...
already saved...
parsing 050121173310.txt...
already saved...
parsing 51020131142.txt...
already saved...
parsing 091120113036.txt...
already saved...
parsing 51020131203.txt...
already saved...
parsing 51020131148.txt...
already saved...
parsing 291020230445.txt...
already saved...
parsing 30920174440.txt...
already saved...
parsing 30920174446.txt

## Saving data 

In [ ]:
def save_data(embeddings, genericity_labels, aspect_labels, boundedness_labels, file_id):
  '''
  Given embeddings, labels, confidence_ratings, and file_id tensors, saves them 
  to a file the name of which is defined using file_id. It is saved in different
  folders if it is from the sitent dataset or not. 
  '''

  # reshape embeddings into storeable shape
  embeddings = embeddings.numpy().reshape(embeddings.shape[0], -1)
  # genericity labels
  genericity_labels = genericity_labels.numpy()

  # if the data to be saved is from Sitent
  if DATASET=='sitent':

    # save embeddings
    h5f = h5py.File(MODEL_DIRECTORY+'/inputs/sitent/sitent_embeddings_' + file_id + '.h5', 'w')
    h5f.create_dataset('embeddings', data=embeddings)
    h5f.close()

    # save genericity labels
    h5f = h5py.File(MODEL_DIRECTORY+'/labels/sitent_genericity/sitent_genericity_labels_' + file_id + '.h5', 'w')
    h5f.create_dataset('genericity_labels', data=genericity_labels)
    h5f.close()

    # save aspect labels
    aspect_labels = aspect_labels.numpy()
    h5f = h5py.File(MODEL_DIRECTORY+'/labels/sitent_aspect/sitent_aspect_labels_' + file_id + '.h5', 'w')
    h5f.create_dataset('aspect_labels', data=aspect_labels)
    h5f.close()
  
  # if the data to be saved is from Reddit glove 
  else:
    # save embeddings
    h5f = h5py.File(MODEL_DIRECTORY+'/inputs/reddit_glove/reddit_glove_embeddings_' + file_id + '.h5', 'w')
    h5f.create_dataset('embeddings', data=embeddings)
    h5f.close()
    
    # save genericity labels
    h5f = h5py.File(MODEL_DIRECTORY+'/labels/reddit_glove_genericity/reddit_glove_genericity_labels_' + file_id  + '.h5', 'w')
    h5f.create_dataset('genericity_labels', data=genericity_labels)
    h5f.close()

    # save aspect labels
    aspect_labels = aspect_labels.numpy()
    h5f = h5py.File(MODEL_DIRECTORY+'/labels/reddit_glove_aspect/reddit_glove_aspect_labels_' + file_id  + '.h5', 'w')
    h5f.create_dataset('aspect_labels', data=aspect_labels)
    h5f.close()

    # save boundedness labels
    boundedness_labels = boundedness_labels.numpy()

    h5f = h5py.File(MODEL_DIRECTORY+'/labels/reddit_glove_boundedness/reddit_glove_boundedness_labels_' + file_id  + '.h5', 'w')
    h5f.create_dataset('boundedness_labels', data=boundedness_labels)
    h5f.close()

#Sitent

https://github.com/annefried/sitent/tree/master/annotated_corpus

parse_and_save_sitent(directory)

In [ ]:
def parse_and_save_sitent(directory):
  '''
  Parses and saves embeddings, labels for sitent 
  data in directory specified by the param.
  @param directory: the directory with .xml files of the sitent dataset
  '''
  for xmlfile in os.listdir(directory):

      file_id = xmlfile.split('.')[0]

      if ('sitent_aspect_labels_' + file_id + '.h5') in os.listdir(MODEL_DIRECTORY + '/labels/sitent_aspect/'):
        print(file_id + ' already saved!')
        continue

      embeddings = []
      genericity_labels = []
      aspect_labels = []
      confidence_ratings = []

      tree = ET.parse(directory + xmlfile) 

      # get root element 
      root = tree.getroot() 
      print('parsing ' + xmlfile + '...')

      for segment in root:
        
        # get genericity label
        genercity_label = -1

        if segment.find('annotation').get('mainReferentGenericity') == 'GENERIC':
          genericity_label = 0
        elif segment.find('annotation').get('mainReferentGenericity') == 'NON-GENERIC':
          genericity_label = 1
        elif segment.find('annotation').get('mainReferentGenericity') == 'CANNOT_DECIDE':
          genericity_label = 2
        else:
          continue

        # get aspect label
        aspect_label = -1

        if segment.find('annotation').get('mainVerbAspectualClass') == 'DYNAMIC':
          aspect_label = 0
        elif segment.find('annotation').get('mainVerbAspectualClass') == 'STATIVE':
          aspect_label = 1
        elif segment.find('annotation').get('mainVerbAspectualClass') == 'CANNOT_DECIDE':
          aspect_label = 2
        else:
          continue
        
        if len(segment[0].text) > 70:
          continue

        clause_embedding = make_embeddings(segment[0].text, TOKENIZER, MODEL, pos=True, ne=True)

        embeddings.append(clause_embedding)
        genericity_labels.append(genericity_label)
        aspect_labels.append(aspect_label)

      embeddings = (tf.stack(embeddings))
      genericity_labels = (tf.stack(genericity_labels))
      aspect_labels = (tf.stack(aspect_labels))

      if (embeddings.shape[0]) == 0:
          continue
      save_data(embeddings, genericity_labels, aspect_labels, file_id, sitent=True, aspect=True)

      print(file_id + ' saved! - ', embeddings.shape, genericity_labels.shape, aspect_labels.shape)
    
# parse_and_save_sitent(SITENT_DIRECTORY)

# MODEL ^_^

## Getting input from saved files

In [ ]:
def get_input_data(files_read=[], reddit_glove_without_muskaan=False):

  if TEMPORAL: 
    feature_size = 809 
    features_directory = '/inputs/reddit_glove_temporal/'
  else: 
    feature_size = 805
    features_directory = '/inputs/reddit_glove/'
  curr_embeddings = tf.zeros([1,70,feature_size])

  curr_labels = tf.zeros([1])

  if reddit_glove_without_muskaan:
    input_directory = '/inputs/reddit_glove_without_muskaan/'
  else:
    input_directory = features_directory

  if DATASET=='reddit':
    print('GETTING INPUTS FROM REDDIT SAVED DATA\n')
    for filename in os.listdir(MODEL_DIRECTORY+input_directory):

      file_id = filename.split('.')[0].split('_')[3]
      print('reading in file ' + file_id + '...')

      if len(filename.split('.')[0].split('_')) > 4:
        file_id += '_' + filename.split('.')[0].split('_')[4]

      # Check if file has already been read before 
      # if filename in files_read:
      #   print(filename + ' already read!')
      #   continue

      if filename in files_read:
        print(filename + ' already read!')
        continue
      
      if curr_embeddings.shape[0] > 5000:
        break
      
      files_read.append(filename)

      # read embeddings
      h5f = h5py.File(MODEL_DIRECTORY+input_directory+'reddit_glove_embeddings_' + file_id + '.h5','r')
      clause_embeddings = h5f['embeddings'][:].reshape((-1,70,feature_size))
      h5f.close()

      # read labels based on the classification task
      if CLASSIFICATION_TYPE=='genericity':
        h5f = h5py.File(MODEL_DIRECTORY+'/labels/reddit_glove_genericity/reddit_glove_genericity_labels_' + file_id + '.h5','r')
        clause_labels = h5f['genericity_labels'][:]
        h5f.close()
      elif CLASSIFICATION_TYPE=='boundedness':
        h5f = h5py.File(MODEL_DIRECTORY+'/labels/reddit_glove_boundedness/reddit_glove_boundedness_labels_' + file_id + '.h5','r')
        clause_labels = h5f['boundedness_labels'][:]
        h5f.close()     
      elif CLASSIFICATION_TYPE == 'aspect':
        h5f = h5py.File(MODEL_DIRECTORY+'/labels/reddit_glove_aspect/reddit_glove_aspect_labels_' + file_id + '.h5','r')
        clause_labels = h5f['aspect_labels'][:]
        h5f.close() 

      if clause_embeddings.shape[0] != clause_labels.shape[0]:
        continue

      new_embeddings = tf.concat([clause_embeddings, curr_embeddings], axis=0)
      curr_embeddings = new_embeddings

      new_labels = tf.concat([clause_labels, curr_labels], axis=0)
      curr_labels = new_labels

      print(curr_embeddings.shape, curr_labels.shape)

  if DATASET=='sitent':
    print('GETTING INPUTS FROM SITENT SAVED DATA\n')
    for filename in os.listdir(MODEL_DIRECTORY+'/inputs/sitent'):

      if filename in files_read:
        print(filename + ' already read!')
        continue
      
      if curr_embeddings.shape[0] > 5000:
        break
      
      files_read.append(filename)

      file_id = filename.split('.')[0].split('sitent_embeddings_')[1]

      h5f = h5py.File(MODEL_DIRECTORY+'/inputs/sitent/sitent_embeddings_' + file_id + '.h5','r')
      clause_embeddings = h5f['embeddings'][:].reshape((-1,70,feature_size))
      h5f.close()

      if CLASSIFICATION_TYPE=='aspect':
        h5f = h5py.File(MODEL_DIRECTORY+'/labels/sitent_aspect/sitent_aspect_labels_' + file_id + '.h5','r')
        clause_labels = h5f['aspect_labels'][:]
        h5f.close()
      elif CLASSIFICATION_TYPE=='genericity':
        h5f = h5py.File(MODEL_DIRECTORY+'/labels/sitent_genericity/sitent_genericity_labels_' + file_id + '.h5','r')
        clause_labels = h5f['genericity_labels'][:]
        h5f.close()  


      new_embeddings = tf.concat([clause_embeddings, curr_embeddings], axis=0)
      curr_embeddings = new_embeddings

      new_labels = tf.concat([clause_labels, curr_labels], axis=0)
      curr_labels = new_labels

      print(curr_embeddings.shape, curr_labels.shape)

  x_train, x_test, y_train, y_test = train_test_split(curr_embeddings.numpy(), curr_labels.numpy(), test_size=0.1, shuffle=True)

  # Convert to tensors
  train_inputs = tf.convert_to_tensor(x_train)
  train_labels = tf.convert_to_tensor(y_train, dtype='int32')
  test_inputs = tf.convert_to_tensor(x_test)
  test_labels = tf.convert_to_tensor(y_test, dtype='int32')

  return train_inputs, train_labels, test_inputs, test_labels, files_read

## Metrics functions - recall, precision, f1

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
num_labels = NUM_LABELS

def get_label_measures(y_true, y_pred):
  y_pred_arr = y_pred.numpy()
  y_true_arr = y_true.numpy()
  predictions = np.zeros([len(y_pred_arr), 1])

  for i in range(0, len(y_pred_arr)):
    predictions[i] = np.argmax(y_pred_arr[i])

  label_measures = []
  for value in range(num_labels):
    label_measures.append(dict())
    
  for label in range(num_labels):
    for index, correct_results in enumerate(y_true_arr):
      correct_results = y_true_arr[index]

      if correct_results[label] == 1 and predictions[index] == label:
        if 'tp' in label_measures[label]:
          label_measures[label]['tp'] += 1
        else:
          label_measures[label]['tp'] = 1
      elif correct_results[label] == 1:
        if 'fn' in label_measures[label]:
          label_measures[label]['fn'] += 1
        else:
          label_measures[label]['fn'] = 1
      elif correct_results[label] != 1 and predictions[index] != label:
        if 'tn' in label_measures[label]:
          label_measures[label]['tn'] + 1
        else:
          label_measures[label]['tn'] = 1
      elif correct_results[label] != 1 and predictions[index] == label:
        if 'fp' in label_measures[label]:
          label_measures[label]['fp'] += 1
        else:
          label_measures[label]['fp'] = 1

  return label_measures

def get_tp_fp_fn(label_measures):
  tps = []
  fns = []
  fps = []
  tns = []
  for label in range(num_labels):
    try:
      tp = label_measures[label]['tp']
    except KeyError:
      tp = 0
    try:
      fp = label_measures[label]['fp']
    except KeyError:
      fp = 0
    try:
      fn = label_measures[label]['fn']
    except KeyError:
      fn = 0
    try:
      tn = label_measures[label]['tn']
    except KeyError:
      tn = 0
    fps.append(fp)
    tps.append(tp)
    fns.append(fn)
    tns.append(tn)
  return tps, fps, fns, tns

def recall_labels(y_true, y_pred):
  label_measures = get_label_measures(y_true, y_pred)
  tps, _, fns, _ = get_tp_fp_fn(label_measures)
  # Return micro-average
  return sum(tps) / (sum(tps) + sum(fns))
    
def precision_labels(y_true, y_pred):
  label_measures = get_label_measures(y_true, y_pred)
  tps, fps, _, _ = get_tp_fp_fn(label_measures)
  # Return the micro-average
  return sum(tps)/(sum(tps) + sum(fps))

def f1_labels(y_true, y_pred):
  micro_p = precision_labels(y_true, y_pred)
  micro_r = recall_labels(y_true, y_pred)
  if (micro_p + micro_r) == 0:
      micro_p += 0.0001
  return 2 * ((micro_p * micro_r) / (micro_p + micro_r)) 


## Model Architecture

In [ ]:
def build_model(num_labels): 

  model = keras.Sequential()

  if TEMPORAL: feature_size = 809
  else: feature_size = 805

  # Add a GRU layer 
  model.add(layers.Bidirectional(layers.GRU(500, return_sequences=True), input_shape=(70, feature_size)))

  model.add(layers.Dropout(0.2))

  # Add another GRU layer  
  model.add(layers.Bidirectional(layers.GRU(300)))

  if num_labels == 2:
    model.add(layers.Dense(2, activation='sigmoid'))
  elif num_labels == 3:
    model.add(layers.Dense(3, activation='softmax'))

  model.summary()

  return model

## Training and testing!

In [ ]:
from collections import Counter

def minority_oversampling(X, y):
  ros = RandomOverSampler(sampling_strategy='minority')
  X = X.numpy().reshape(X.shape[0], -1)
  X_resampled, y_resampled = ros.fit_resample(X, y)
  
  if TEMPORAL: feature_size = 809
  else: feature_size = 805

  X_resampled = X_resampled.reshape(-1, 70, feature_size)

  print(Counter(y.numpy()))
  print(Counter(y_resampled))
  print(X_resampled.shape)

  return X_resampled, y_resampled


In [ ]:
def train_and_test(model, batch_size, epochs, num_labels):
  '''
  Train the model using the inputs, labels, batch_size, and epochs.
  '''
  # Create a callback that saves the model's weights
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=CHECKPOINT_PATH,
                                                  frequency = "epoch",
                                                  save_weights_only=False,
                                                  verbose=1)
  
  # check if valid training params
  if (DATASET=='sitent' and CLASSIFICATION_TYPE=='boundedness'):
    print('Sorry, cannot train boundedness on the sitent dataset')
    return 

  all_test_inputs = []
  all_test_labels = []
  files_read = []
  
  if DATASET=='reddit':
    for file_batch_count in range(4):

    # get Reddit glove inputs 
      reddit_glove_train_inputs, reddit_glove_train_labels, reddit_glove_test_inputs, reddit_glove_test_labels, files_read = get_input_data()
      print(reddit_glove_train_inputs.shape, reddit_glove_train_labels.shape)

      # minority oversampling!
      # reddit_glove_train_inputs, reddit_glove_train_labels = minority_oversampling(reddit_glove_train_inputs, reddit_glove_train_labels)

      reddit_glove_train_labels = (tf.one_hot(reddit_glove_train_labels, num_labels))
      reddit_glove_test_labels = (tf.one_hot(reddit_glove_test_labels, num_labels))

      all_test_inputs.append(reddit_glove_test_inputs)
      all_test_labels.append(reddit_glove_test_labels)

      # class weights!
      y_train = [y.numpy().argmax() for y in reddit_glove_train_labels]
      class_weights = class_weight.compute_class_weight('balanced', 
                                                        np.unique(y_train),
                                                        y_train)
      class_weights_dict = {}
      for index, weight in enumerate(class_weights):
        class_weights_dict[index] = weight

      print('Class weights: ', class_weights_dict)
      print(reddit_glove_train_labels)

      # FIT REDDIT GLOVE
      model.fit(reddit_glove_train_inputs, reddit_glove_train_labels, batch_size, epochs, 
                validation_split=0.1, callbacks=[cp_callback], class_weight=class_weights_dict)
                #callbacks=[cp_callback], 
    

  sitent_files_read = []
  if DATASET=='sitent':
    for file_batch_count in range(7):
      # get Sitent glove inputs 
      sitent_train_inputs, sitent_train_labels, sitent_test_inputs, sitent_test_labels, files_read = get_input_data(files_read=sitent_files_read)
      print(sitent_train_inputs.shape, sitent_train_labels.shape)

      # minority oversampling!
      # sitent_train_inputs, sitent_train_labels = minority_oversampling(sitent_train_inputs, sitent_train_labels)

      sitent_train_labels = (tf.one_hot(sitent_train_labels, num_labels))
      sitent_test_labels = (tf.one_hot(sitent_test_labels, num_labels))        

      all_test_inputs.append(sitent_test_inputs)
      all_test_labels.append(sitent_test_labels)

      sitent_files_read = files_read

      # class weights!
      y_train = [y.numpy().argmax() for y in sitent_train_labels]
      class_weights = class_weight.compute_class_weight('balanced', 
                                                        np.unique(y_train),
                                                        y_train)
      class_weights_dict = {}
      for index, weight in enumerate(class_weights):
        class_weights_dict[index] = weight

      print('Class weights: ', class_weights_dict)
      
      # FIT SITENT
      model.fit(sitent_train_inputs, sitent_train_labels, batch_size, epochs, 
                validation_split=0.1, callbacks=[cp_callback], class_weight=class_weights_dict)

  # TEST 
  print("Evaluate on test data")
  test_inputs = tf.concat(all_test_inputs,axis=0)
  test_labels = tf.argmax(tf.concat(all_test_labels,axis=0),axis=1)
  
  y_pred = (tf.argmax(model.predict(test_inputs),axis=1))
  print(classification_report(test_labels, y_pred))

  # results = model.evaluate(all_test_inputs, all_test_labels, batch_size)
  # print("test loss, test acc:", results)


# Checking Label Specific Metrics for Sitent

In [ ]:
def train_reddit_sitent():

  model_path = "/content/drive/MyDrive/Clause Classification Models/saved/" + MODEL_NAME
  model = load_model(model_path, custom_objects={'f1_m':f1_m,'recall_m':recall_m,'precision_m':precision_m})
  opt = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(
  loss=keras.losses.CategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  run_eagerly=True,
  metrics=['acc', keras.metrics.Precision(), recall_m, f1_m],)  

  # TRAIN 
  train_and_test(model, BATCH_SIZE, EPOCHS, 3)

  model.save(model_path)

BATCH_SIZE = 32
EPOCHS = 1
MODEL_NAME = 'final_reddit_sitent_aspect_32_1_cw.ckpt'


DATASET = 'reddit' # sitent or reddit
CLASSIFICATION_TYPE = 'aspect' # genericity, aspect, or boundedness
train_reddit_sitent()

GETTING INPUTS FROM REDDIT SAVED DATA

reading in file 091120113028...
(225, 70, 805) (225,)
reading in file 291020230505...
(294, 70, 805) (294,)
reading in file 30920174438...
(345, 70, 805) (345,)
reading in file 091120113041...
(407, 70, 805) (407,)
reading in file 091120113025...
(458, 70, 805) (458,)
reading in file 291020230442...
(500, 70, 805) (500,)
reading in file 51020131148...
(522, 70, 805) (522,)
reading in file 30920174446...
(579, 70, 805) (579,)
reading in file 51020131203...
(606, 70, 805) (606,)
reading in file 51020131202...
(682, 70, 805) (682,)
reading in file 51020131206...
(749, 70, 805) (749,)
reading in file 51020131151...
(819, 70, 805) (819,)
reading in file 51020131153...
(877, 70, 805) (877,)
reading in file 51020131143...
(958, 70, 805) (958,)
reading in file 51020131156...
(1013, 70, 805) (1013,)
reading in file 51020131142...
(1044, 70, 805) (1044,)
reading in file 291020230455...
(1089, 70, 805) (1089,)
reading in file 291020230445...
(1153, 70, 805) 

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4870: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`categorical_crossentropy` received `from_logits=True`, but '


254/254 [==============================] - 4352s 17s/step - loss: 0.4084 - acc: 0.8130 - precision: 0.8130 - recall_m: 0.8129 - f1_m: 0.8129 - val_loss: 0.4042 - val_acc: 0.8248 - val_precision: 0.8248 - val_recall_m: 0.8204 - val_f1_m: 0.8204

Epoch 00001: saving model to /content/drive/MyDrive/Clause Classification Models/saved/final_reddit_sitent_genericity_16_1_cw.ckpt


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Clause Classification Models/saved/final_reddit_sitent_genericity_16_1_cw.ckpt/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Clause Classification Models/saved/final_reddit_sitent_genericity_16_1_cw.ckpt/assets


...

Evaluate on test data


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.79      0.84      0.81       907
           1       0.84      0.79      0.82       971
           2       0.00      0.00      0.00         0

    accuracy                           0.81      1878
   macro avg       0.54      0.54      0.54      1878
weighted avg       0.82      0.81      0.81      1878



INFO:tensorflow:Assets written to: /content/drive/MyDrive/Clause Classification Models/saved/final_reddit_sitent_genericity_16_1_cw.ckpt/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Clause Classification Models/saved/final_reddit_sitent_genericity_16_1_cw.ckpt/assets


In [ ]:
def test_saved_model(path, reddit_glove_without_muskaan=False):
  model = load_model(path, custom_objects={'f1_m':f1_m,'recall_m':recall_m,'precision_m':precision_m})
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', f1_m])

  all_test_labels = []
  all_y_pred = []
  files_read = []

  for i in range(8):
    inputs, labels, test_inputs, test_labels, files_read = get_input_data(files_read=files_read, reddit_glove_without_muskaan=reddit_glove_without_muskaan)

    test_inputs = tf.concat([inputs, test_inputs], axis=0)
    test_labels = tf.concat([labels, test_labels], axis=0)

    y_pred = (tf.argmax(model.predict(test_inputs),axis=1))
    all_test_labels.append(test_labels)
    all_y_pred.append(y_pred)
    print(y_pred, test_labels)

  all_test_labels = tf.concat(all_test_labels, axis=0)
  all_y_pred = tf.concat(all_y_pred, axis=0)
  print(classification_report(all_test_labels, all_y_pred))


DATASET = 'sitent'
CLASSIFICATION_TYPE = 'aspect'
NUM_LABELS = 2
TEMPORAL = False
model_name = 'final_reddit_sitent_aspect_32_1_cw.ckpt'
model_path = "/content/drive/MyDrive/Clause Classification Models/saved/" + model_name
test_saved_model(CHECKPOINT_PATH, reddit_glove_without_muskaan=False)


# Main

This is the main function that will train a model, evaluate it, and save it based on the global variables declared.



In [ ]:
def main(): 

  num_labels = 2

  if DATASET == 'sitent':
    if CLASSIFICATION_TYPE == 'genericity':
      num_labels = 3
    elif CLASSIFICATION_TYPE == 'aspect':
      num_labels = 3
    elif CLASSIFICATION_TYPE == 'boundedness':
      print('Invalid CLASSIFICATION_TYPE! Cannot train boundedness on sitent.')
    else:
      print('Invalid CLASSIFICATION_TYPE! Use genericity, aspect, or boundedness.')

  elif DATASET == 'reddit':
    if CLASSIFICATION_TYPE == 'genericity':
      num_labels = 2
    elif CLASSIFICATION_TYPE == 'aspect':
      num_labels = 2
    elif CLASSIFICATION_TYPE == 'boundedness':
      num_labels = 2
    else:
      print('Invalid CLASSIFICATION_TYPE! Use genericity, aspect, or boundedness.')

  else:
    print('Invalid DATASET! Use sitent or reddit.')

  # build model 
  model = build_model(num_labels)

  opt = keras.optimizers.Adam(learning_rate=0.001)

  if num_labels == 2:

    model.compile(
      loss=keras.losses.BinaryCrossentropy(from_logits=True),
      optimizer=opt,
      run_eagerly=True,
      #metrics=['acc', precision_labels, recall_labels, f1_labels],)
      metrics=['acc', keras.metrics.Precision(), recall_m, f1_m],)

  elif num_labels == 3:

    model.compile(
      loss=keras.losses.CategoricalCrossentropy(from_logits=True),
      optimizer=opt,
      run_eagerly=True,
      #metrics=['acc', precision_labels, recall_labels, f1_labels],)
      metrics=['acc', keras.metrics.Precision(), recall_m, f1_m],)

  # TRAIN 
  train_and_test(model, BATCH_SIZE, EPOCHS, num_labels)

  model.save(CHECKPOINT_PATH)

if __name__ == '__main__':
    main()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 70, 1000)          3921000   
_________________________________________________________________
dropout_75 (Dropout)         (None, 70, 1000)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 600)               2343600   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1202      
Total params: 6,265,802
Trainable params: 6,265,802
Non-trainable params: 0
_________________________________________________________________
GETTING INPUTS FROM REDDIT SAVED DATA

reading in file 091120113028...
(225, 70, 805) (225,)
reading in file 291020230505...
(294, 70, 805) (294,)
reading in file 30920174438...
(345, 70, 805) (345,)
reading in file 091120113041...
(407, 70, 

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:5017: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


127/127 [==============================] - 1202s 9s/step - loss: 0.5742 - acc: 0.7272 - precision_1: 0.7265 - recall_m: 0.7209 - f1_m: 0.7144 - val_loss: 0.4369 - val_acc: 0.7960 - val_precision_1: 0.7942 - val_recall_m: 0.7882 - val_f1_m: 0.7873

Epoch 00001: saving model to /content/drive/MyDrive/Clause Classification Models/saved/final_reddit_sitent_genericity_64_3_cw.ckpt


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Clause Classification Models/saved/final_reddit_sitent_genericity_64_3_cw.ckpt/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Clause Classification Models/saved/final_reddit_sitent_genericity_64_3_cw.ckpt/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Clause Classification Models/saved/final_reddit_sitent_genericity_64_3_cw.ckpt/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Clause Classification Models/saved/final_reddit_sitent_genericity_64_3_cw.ckpt/assets


Evaluate on test data
              precision    recall  f1-score   support

           0       0.89      0.79      0.84      1058
           1       0.77      0.88      0.82       820

    accuracy                           0.83      1878
   macro avg       0.83      0.83      0.83      1878
weighted avg       0.84      0.83      0.83      1878



INFO:tensorflow:Assets written to: /content/drive/MyDrive/Clause Classification Models/saved/final_reddit_sitent_genericity_64_3_cw.ckpt/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Clause Classification Models/saved/final_reddit_sitent_genericity_64_3_cw.ckpt/assets


# Model Analysis

After implementing the infrastructure needed to train a bunch of different models easily on either dataset, I went on to compare and contrast

### Genericity 

---


Trained on: Sitent\
Test data: Sitent\
Type: Genericity\
Batch Size: 64\
Epochs: 1

```
precision    recall  f1-score   support

0       0.86      0.48      0.62       266
1       0.85      0.96      0.90      1214
2       0.93      0.88      0.91       567

accuracy 0.87      2047
macro avg 0.88      0.77      0.81      2047
weighted avg 0.88      0.87      0.87      2047
```

Trained on: Sitent\
Test data: Reddit\
Type: Genericity\
Batch Size: 64\
Epochs: 1

```
precision    recall  f1-score   support

0       0.90      0.10      0.17      3925
1       0.56      0.94      0.70      4167
2       0.00      0.00      0.00         0

accuracy  0.53      8092
macro avg  0.49      0.35      0.29      8092
weighted avg 0.72      0.53      0.44      8092
```
####**BEST ONE**
Trained on: Sitent\
Test data: Reddit\
Type: Genericity\
Batch Size: 64\
Epochs: 1\
Class Weights 

```
precision    recall  f1-score   support

0       0.80      0.58      0.67      3925
1       0.74      0.79      0.77      4167
2       0.00      0.00      0.00         0

accuracy  0.69      8092
macro avg 0.52      0.46      0.48      8092
weighted avg 0.77      0.69      0.72      8092

```

####Reddit Training

Trained on: Reddit\
Test data: Reddit\
Type: Genericity\
Batch Size: 64\
Epochs: 1

```
precision    recall  f1-score   support

0       0.79      0.82      0.81       388
1       0.83      0.79      0.81       422

accuracy  0.81       810
macro avg  0.81      0.81      0.81       810
weighted avg  0.81      0.81      0.81       810
```

Trained on: Reddit\
Test data: Reddit\
Type: Genericity\
Batch Size: 64\
Epochs: 1\
Class Weights

```
precision    recall  f1-score   support

0       0.79      0.70      0.74       407
1       0.73      0.81      0.77       403

accuracy  0.75       810
macro avg  0.76      0.75      0.75       810
weighted avg 0.76      0.75      0.75       810
```

Trained on: Reddit\
Test data: Reddit\
Type: Genericity\
Batch Size: 64\
Epochs: 1\
Minority Oversampling

```
precision    recall  f1-score   support

0       0.74      0.91      0.82       413
1       0.88      0.68      0.76       397

accuracy  0.80       810
macro avg  0.81      0.79      0.79       810
weighted avg 0.81      0.80      0.79       810
```

### Aspect

TODO: Correct macro average, include muskaan!
---

Trained on: Sitent and Reddit \
Test data: Reddit\
Type: Aspect\
Batch Size: 16\
Epochs: 1\
Class Weights

```
precision    recall  f1-score   support

0       0.86      0.85      0.85       460
1       0.81      0.81      0.81       350

accuracy  0.83       810
macro avg  0.83      0.83      0.83       810
weighted avg 0.83      0.83      0.83       810
```


Trained on: Sitent\
Test data: Sitent\
Type: Aspect\
Batch Size: 16\
Epochs: 1\
Class Weights
```
    precision    recall  f1-score   support

0       0.84      0.93      0.88       850
1       0.92      0.78      0.85       679
2       0.92      0.93      0.92       518

accuracy 0.88      2047
macro avg  0.89      0.88      0.88      2047
weighted avg 0.89      0.88      0.88      2047
```
Trained on: Sitent\
Test data: Sitent\
Type: Aspect\
Batch Size: 64\
Epochs: 1

```
  precision    recall  f1-score   support

0       0.88      0.88      0.88      1980
1       0.84      0.87      0.85      1540
2       0.95      0.91      0.93      1576

accuracy                    0.89      5096
macro avg  0.89      0.89      0.89      5096
weighted avg  0.89      0.89      0.89      5096
```

Trained on: Sitent\
Test data: Reddit\
Type: Aspect\
Batch Size: 64\
Epochs: 1


```
    precision    recall  f1-score   support

0       0.84      0.73      0.78      4623
1       0.77      0.76      0.77      3469
2       0.00      0.00      0.00         0

accuracy                    0.75      8092
macro avg  0.54      0.50      0.52      8092
weighted avg 0.81      0.75      0.78      8092
```

Trained on: Sitent\
Test data: Reddit without Muskaan\
Type: Aspect\
Batch Size: 64\
Epochs: 1

```
    precision    recall  f1-score   support

0       0.86      0.74      0.80      3941
1       0.76      0.77      0.77      2720
2       0.00      0.00      0.00         0

accuracy                    0.76      6661
macro avg  0.54      0.51      0.52      6661
weighted avg 0.82      0.76      0.78      6661
```








### Boundedness
---


Trained on: Reddit\
Test data: Reddit\
Type: Boundedness\
Batch Size: 16\
Epochs: 2\
Temporal Features
```
precision    recall  f1-score   support

0       0.82      0.72      0.77       292
1       0.88      0.93      0.90       639

accuracy 0.86       931
macro avg 0.85      0.82      0.83       931
weighted avg 0.86      0.86      0.86       931
```

####**USE THIS**

Trained on: Reddit\
Test data: Reddit\
Type: Boundedness\
Batch Size: 32\
Epochs: 2\
Temporal Features

```
precision    recall  f1-score   support

0       0.85      0.72      0.78       299
1       0.88      0.94      0.91       632

accuracy  0.87       931
macro avg  0.86      0.83      0.84       931
weighted avg 0.87      0.87      0.87       931
```

Trained on: Reddit\
Test data: Reddit\
Type: Boundedness\
Batch Size: 64\
Epochs: 1\
Temporal Features
```
precision    recall  f1-score   support

0   0.77      0.73      0.75       308
1   0.87      0.89      0.88       623

accuracy 0.84       931
macro avg  0.82      0.81      0.81       931
weighted avg 0.84      0.84      0.84       931
```

Trained on: Reddit\
Test data: Reddit\
Type: Boundedness\
Batch Size: 64\
Epochs: 1\
Temporal Features
Minority Oversampling

```
precision    recall  f1-score   support

0       0.78      0.69      0.73       288
1       0.87      0.91      0.89       643

accuracy 0.84       931
macro avg 0.82      0.80      0.81       931
weighted avg 0.84      0.84      0.84       931
```

Trained on: Reddit\
Test data: Reddit\
Type: Boundedness\
Batch Size: 64\
Epochs: 1\
Class Weights

```
precision    recall  f1-score   support

0       0.66      0.72      0.69       229
1       0.88      0.86      0.87       581

accuracy 0.82       810
macro avg 0.77      0.79      0.78       810
weighted avg 0.82      0.82      0.82       810
```
Trained on: Reddit\
Test data: Reddit\
Type: Boundedness\
Batch Size: 64\
Epochs: 1\
Minority Oversampling
 
```
Evaluate on test data
precision    recall  f1-score   support

0       0.63      0.90      0.74       50
1       0.95      0.76      0.84       560

accuracy  0.81       810
macro avg  0.79      0.83      0.79       810
weighted avg 0.85      0.81      0.81       810
```


# Final Analyses

## Genericity Model

**USED**\
Trained on: Sitent\
Test data: Reddit FULL\
Type: Genericity\
Batch Size: 64\
Epochs: 1\
Class Weights 

```
precision    recall  f1-score   support

0       0.82      0.69      0.75      9984
1       0.78      0.78      0.78     10060
2       0.00      0.00      0.00         0

accuracy                           0.74     20044
macro avg       0.54      0.49      0.51     20044
weighted avg       0.80      0.74      0.77     20044
```

Trained on: Sitent\
Test data: Sitent\
Type: Genericity\
Batch Size: 64\
Epochs: 1\
Class Weights 

```
precision    recall  f1-score   support

0       0.61      0.70      0.66       506
1       0.93      0.84      0.88      2189
2       0.83      0.95      0.89       869

accuracy                           0.85      3564
macro avg       0.79      0.83      0.81      3564
weighted avg       0.86      0.85      0.85      3564
```

Trained on: Reddit FULL\
Test data: Reddit FULL\
Type: Genericity\
Batch Size: 64\
Epochs: 1\
Class Weights 

```
precision    recall  f1-score   support

0       0.85      0.73      0.79       927
1       0.77      0.88      0.82       951

accuracy                           0.81      1878
macro avg       0.81      0.80      0.80      1878
weighted avg       0.81      0.81      0.80      1878
```
Trained on: Reddit FULL\
Test data: Sitent\
Type: Genericity\
Batch Size: 64\
Epochs: 1\
Class Weights 

```
0       0.34      0.68      0.45      6828
1       0.62      0.84      0.72     35172
2       0.00      0.00      0.00     19152

accuracy                           0.56     61152
macro avg       0.32      0.51      0.39     61152
weighted avg       0.40      0.56      0.46     61152

```

Trained on: Sitent + Reddit\
Test data: Reddit\
Type: Genericity\
Batch Size: 64\
Epochs: 1\
Class Weights 

```
precision    recall  f1-score   support

0       0.75      0.92      0.83      9350
1       0.89      0.70      0.79      9404

accuracy                           0.81     18754
macro avg       0.82      0.81      0.81     18754
weighted avg       0.82      0.81      0.81     18754
```

Trained on: Sitent + Reddit\
Test data: Sitent\
Type: Genericity\
Batch Size: 64\
Epochs: 1\
Class Weights 

```
precision    recall  f1-score   support

0       0.29      0.94      0.44      5700
1       0.75      0.64      0.69     23504
2       0.00      0.00      0.00      9484

accuracy                           0.53     38688
macro avg       0.35      0.53      0.38     38688
weighted avg       0.50      0.53      0.48     38688
```


## Aspect 

Trained on: Sitent + Reddit\
Test data: Reddit\
Type: Aspect\
Batch Size: 32\
Epochs: 1\
Class Weights 

```
precision    recall  f1-score   support

0       0.93      0.75      0.83      1029
1       0.75      0.93      0.83       849

accuracy                           0.83      1878
macro avg       0.84      0.84      0.83      1878
weighted avg       0.85      0.83      0.83      1878
```